In [1]:
%cd /Users/yantong/Desktop/resume.md-main/Project/AirBnB/Project

/Users/yantong/Desktop/resume.md-main/Project/AirBnB/Project


In [59]:
# Import our libraries we are going to use for our data analysis.
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb

In [276]:
from sklearn.metrics import confusion_matrix


In [277]:
# Show all the coplumns
pd.set_option('display.max_columns', None)
# show all the rows
pd.set_option('display.max_rows', None)
# Set the value length as 100
pd.set_option('max_colwidth',100)


pd.set_option("display.max_colwidth",1000)
pd.set_option('display.width',1000)

pd.set_option('max_info_rows', None)

#To plot figs on jupyter
%matplotlib inline

In [383]:
train_data = pd.read_csv('train_data.csv')
valid_data = pd.read_csv('valid_data.csv')

In [384]:
train_data.head()

,host_since,host_location,host_response_time,host_response_rate,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,zipcode,is_location_exact,room_type,accommodates,bedrooms,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,reviews_per_month,log_price
0,2012.0,"Berlin, Berlin, Germany",NaN,NaN,Prenzlauer Berg,t,Prenzlauer Berg Südwest,10435.0,t,Entire home/apt,2,0,0,25,1,0,3,1125,1140,0,0,1,2015.0,2015.0,100,8,10,10,f,flexible,1,0.03,3.871201
1,2015.0,"Berlin, Berlin, Germany",NaN,NaN,Prenzlauer Berg,t,Prenzlauer Berg Nordwest,10437.0,t,Private room,2,1,0,0,1,0,4,1125,870,0,0,10,2015.0,2016.0,88,8,10,9,f,moderate,1,0.25,3.555348
2,2017.0,"Berlin, Berlin, Germany",NaN,NaN,NaN,t,Prenzlauer Berg Nordwest,10439.0,f,Entire home/apt,2,1,108,27,2,10,1,365,1,37,149,50,2017.0,2018.0,90,10,8,9,t,super_strict_30,45,2.73,3.737670
3,2013.0,"Berlin, Berlin, Germany",within a day,high,Prenzlauer Berg,f,Helmholtzplatz,10437.0,t,Private room,1,1,0,0,1,0,1,4,60,88,363,8,2015.0,2018.0,95,10,10,9,f,flexible,1,0.19,3.496508
4,2016.0,"Berlin, Berlin, Germany",NaN,NaN,Prenzlauer Berg,t,Prenzlauer Berg Nordwest,10437.0,t,Entire home/apt,2,1,0,60,1,0,7,27,630,0,0,1,2016.0,2016.0,100,10,10,10,f,moderate,1,0.04,4.174387


In [385]:
valid_data.head()

,host_since,host_location,host_response_time,host_response_rate,host_neighbourhood,host_identity_verified,neighbourhood_cleansed,zipcode,is_location_exact,room_type,accommodates,bedrooms,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,calculated_host_listings_count,reviews_per_month,log_price
0,2018,DE,NaN,NaN,Prenzlauer Berg,f,Prenzlauer Berg Nord,10439.0,t,Entire home/apt,2,0,55,200,50,1,0,2,7,1,51,51,10,2018.0,2018.0,94,9,10,9,f,moderate,1,1.24,4.007333
1,2014,"Berlin, Berlin, Germany",within a few hours,high,Prenzlauer Berg,f,Prenzlauer Berg Süd,10407.0,t,Private room,1,1,35,500,15,1,18,2,4,150,87,87,2,2018.0,2018.0,100,10,10,10,f,strict_14_with_grace_period,2,0.22,3.555348
2,2012,"Berlin, Berlin, Germany",NaN,NaN,NaN,f,Prenzlauer Berg Nord,10439.0,f,Private room,2,0,65,0,20,1,0,1,1125,7,9,9,3,2018.0,2018.0,100,10,10,10,f,moderate,1,0.50,4.174387
3,2014,"Berlin, Berlin, Germany",a few days or more,low,Prenzlauer Berg,f,Prenzlauer Berg Nord,10439.0,t,Private room,4,2,95,0,40,1,0,3,13,180,75,165,3,2017.0,2018.0,100,10,9,9,f,strict_14_with_grace_period,1,0.21,4.553877
4,2015,"Stuttgart, Baden-Württemberg, Germany",NaN,NaN,Other,f,Prenzlauer Berg Nord,10439.0,f,Private room,2,1,22,0,0,1,0,2,15,330,0,0,1,2018.0,2018.0,100,8,10,8,t,moderate,2,0.10,3.091042


In [386]:
valid_data.host_since = valid_data.host_since.astype(float)

In [387]:
valid_data.describe()

,host_since,zipcode,accommodates,bedrooms,price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,log_price
count,1108.000000,1083.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,930.000000,931.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000,1108.000000
mean,2014.399819,10423.074792,2.670578,1.189531,71.287004,139.602888,21.606498,1.358303,8.573105,6.375451,572.637184,218.629964,19.259025,78.267148,22.233755,2016.207527,2017.494092,95.025271,9.495487,9.745487,9.522563,1.856498,0.988944,4.011372
std,2.012580,197.452147,1.386071,0.647557,168.208373,337.994384,24.273368,0.788784,12.103106,16.120028,541.683218,320.342548,26.819256,116.633239,44.948173,1.719560,0.953047,6.432141,0.910134,0.527669,0.765349,3.776763,1.416053,0.583845
min,2009.000000,10119.000000,1.000000,0.000000,13.000000,0.000000,0.000000,1.000000,0.000000,1.000000,2.000000,1.000000,0.000000,0.000000,0.000000,2009.000000,2011.000000,20.000000,2.000000,6.000000,2.000000,1.000000,0.000000,2.564949
25%,2013.000000,10405.000000,2.000000,1.000000,38.000000,0.000000,0.000000,1.000000,0.000000,2.000000,20.000000,7.000000,0.000000,0.000000,1.000000,2015.000000,2017.000000,93.000000,9.000000,10.000000,9.000000,1.000000,0.070000,3.637586
50%,2014.000000,10435.000000,2.000000,1.000000,53.000000,0.000000,15.000000,1.000000,5.000000,3.000000,365.000000,60.000000,0.000000,4.000000,5.000000,2017.000000,2018.000000,97.000000,10.000000,10.000000,10.000000,1.000000,0.370000,3.970292
75%,2016.000000,10437.000000,3.000000,1.000000,79.000000,200.000000,35.000000,2.000000,15.000000,4.000000,1125.000000,300.000000,35.000000,128.000000,21.000000,2018.000000,2018.000000,100.000000,10.000000,10.000000,10.000000,1.000000,1.360000,4.369448
max,2018.000000,13189.000000,16.000000,5.000000,5000.000000,4180.000000,250.000000,12.000000,200.000000,183.000000,2000.000000,1740.000000,90.000000,365.000000,498.000000,2018.000000,2018.000000,100.000000,10.000000,10.000000,10.000000,45.000000,10.750000,8.517193


In [388]:
train_data['host_since'] = train_data['host_since'].fillna(2015)

In [389]:
train_data = train_data.drop(["neighbourhood_cleansed","zipcode","first_review",'last_review','host_location'],axis = 1)
valid_data = valid_data.drop(["neighbourhood_cleansed","zipcode","first_review","last_review",'host_location'],axis = 1)


In [390]:
categorial_features = ['host_since','host_response_time','host_response_rate','room_type','cancellation_policy',"host_neighbourhood",
                       'host_identity_verified','is_location_exact','instant_bookable']
train_data_2 = train_data
for categorical_feature in list(train_data_2.columns):
    if categorical_feature in categorial_features:
        feature_dummies = pd.get_dummies(train_data_2[categorical_feature],drop_first=True,prefix=categorical_feature)
        train_data_2 = pd.concat([train_data_2, feature_dummies], axis=1)

In [391]:
categorial_features = ['host_since','host_response_time','host_response_rate','room_type','cancellation_policy',"host_neighbourhood",
                       'host_identity_verified','is_location_exact','instant_bookable']
valid_data_2 = valid_data
for categorical_feature in list(valid_data_2.columns):
    if categorical_feature in categorial_features:
        feature_dummies = pd.get_dummies(valid_data_2[categorical_feature],drop_first=True,prefix=categorical_feature)
        valid_data_2 = pd.concat([valid_data_2, feature_dummies], axis=1)

In [392]:
categorial_features

['host_since',
 'host_response_time',
 'host_response_rate',
 'room_type',
 'cancellation_policy',
 'host_neighbourhood',
 'host_identity_verified',
 'is_location_exact',
 'instant_bookable']

In [393]:
train_data_2 = train_data_2.drop(categorial_features,axis=1)

In [394]:
valid_data_2 = valid_data_2.drop(categorial_features,axis=1)

In [395]:
valid_data_2['host_since_2009.0'] = 0

In [400]:
X_train = train_data_2.drop(["log_price"],axis =1)
y_train = train_data_2["log_price"]

In [401]:
X_test = valid_data_2.drop(["log_price"],axis =1)
y_test = valid_data_2["log_price"]

In [402]:
X_test = X_test.drop(['price'],axis = 1)

In [403]:
X_train.describe()

,accommodates,bedrooms,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,host_since_2009.0,host_since_2010.0,host_since_2011.0,host_since_2012.0,host_since_2013.0,host_since_2014.0,host_since_2015.0,host_since_2016.0,host_since_2017.0,host_since_2018.0,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_low,host_neighbourhood_Prenzlauer Berg,host_identity_verified_t,is_location_exact_t,room_type_Private room,room_type_Shared room,instant_bookable_t,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30
count,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.000000,1656.00000
mean,2.680556,1.182367,131.788043,21.504227,1.382850,8.609903,7.999396,612.183575,227.508454,18.099034,76.785628,21.245169,95.265097,9.536232,9.766304,9.552536,1.874396,0.961721,0.005435,0.018720,0.065821,0.106280,0.151570,0.170894,0.182971,0.141304,0.096014,0.059783,0.104469,0.107488,0.222222,0.056763,0.745169,0.432367,0.744565,0.408213,0.009058,0.280193,0.324879,0.312198,0.00785
std,1.329201,0.638600,307.136911,24.284741,0.896206,11.231484,38.287220,537.066575,332.392882,26.036435,115.329097,44.812334,6.018934,0.843610,0.518282,0.710487,4.087552,1.345307,0.073543,0.135575,0.248044,0.308289,0.358712,0.376530,0.386760,0.348440,0.294700,0.237155,0.305960,0.309826,0.415865,0.231460,0.435898,0.495554,0.436237,0.491651,0.094770,0.449229,0.468471,0.463530,0.08828
min,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,20.000000,2.000000,6.000000,2.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
25%,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000,2.000000,25.750000,6.000000,0.000000,0.000000,1.000000,93.000000,9.000000,10.000000,9.000000,1.000000,0.070000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
50%,2.000000,1.000000,0.000000,16.000000,1.000000,5.000000,3.000000,1124.000000,60.000000,0.000000,4.000000,6.000000,97.000000,10.000000,10.000000,10.000000,1.000000,0.370000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000
75%,4.000000,1.000000,200.000000,35.000000,2.000000,15.000000,5.000000,1125.000000,330.000000,35.000000,121.250000,19.000000,100.000000,10.000000,10.000000,10.000000,1.000000,1.300000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,1.000000,0.00000
max,11.000000,4.000000,4253.000000,200.000000,16.000000,100.000000,1000.000000,1200.000000,1770.000000,90.000000,365.000000,482.000000,100.000000,10.000000,10.000000,10.000000,45.000000,8.670000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.

In [405]:
clf_xgb=xgb.XGBClassifier(random_state=922)
clf_xgb.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=922,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

##### Hyoeroarameters tuning
###### step 1: n_estimators

In [415]:
cv_params = {'n_estimators': [50,55,60,65,70]}
other_params = {'learning_rate': 0.1, 'n_estimators': 500, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

In [416]:
clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)


Fitting 5 folds for each of 5 candidates, totalling 25 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  18 out of  25 | elapsed:    1.3s remaining:    0.5s
[Parallel(n_jobs=4)]: Done  25 out of  25 | elapsed:    1.5s finished


[22:43:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=5,
                                    min_child_weight=1, missing=None,
                                    n_estimators=500, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=0, silent=None,
                                    subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=4,
             param_grid={'n_estimators': [50, 55, 60, 65, 70]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=F

In [417]:
optimized_GBM.best_params_

{'n_estimators': 65}

##### Step 2: max_depth & min_child_weight

In [418]:
cv_params = {'max_depth': list(range(1,5,1)), 'min_child_weight': [1, 2, 3, 4, 5, 6]}
other_params = {'learning_rate': 0.1, 'n_estimators': 65, 'max_depth': 5, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)


Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  76 tasks      | elapsed:    2.9s
[Parallel(n_jobs=4)]: Done 120 out of 120 | elapsed:    5.2s finished


[22:43:54] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


GridSearchCV(cv=5, error_score=nan,
             estimator=XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=0.8, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=5,
                                    min_child_weight=1, missing=None,
                                    n_estimators=65, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=0, silent=None,
                                    subsample=0.8, verbosity=1),
             iid='deprecated', n_jobs=4,
             param_grid={'max_depth': [1, 2, 3, 4],
                         'min_child_weight': [1, 2, 3, 4, 5, 6]},
             pre_

In [419]:
optimized_GBM.best_params_

{'max_depth': 4, 'min_child_weight': 1}

###### Step 3: gamma

In [422]:
cv_params = {'gamma': [ 0.5, 0.6,0.7,0.8]}
other_params = {'learning_rate': 0.1, 'n_estimators': 65, 'max_depth': 4, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0, 'reg_alpha': 0, 'reg_lambda': 1}

clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)
optimized_GBM.best_params_

Fitting 5 folds for each of 4 candidates, totalling 20 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  20 out of  20 | elapsed:    1.1s finished


[22:44:51] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'gamma': 0.6}

###### step 4: subsample, colsample_bytree：

In [425]:
cv_params = {'subsample': [ 0.7,0.75,0.8,0.85], 'colsample_bytree': [ 0.7, 0.75,0.8,0.85]}
other_params = {'learning_rate': 0.1, 'n_estimators': 65, 'max_depth': 4, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.8, 'gamma': 0.6, 'reg_alpha': 0, 'reg_lambda': 1}

clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)
optimized_GBM.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.2s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:    4.4s finished


[22:45:42] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'colsample_bytree': 0.85, 'subsample': 0.8}

###### step 5: regalpha & reglambda

In [427]:
cv_params = {'reg_alpha': [ 0.1,0.2,0.3, 0.5], 'reg_lambda': [0.01,0.05, 0.1,0.2]}
other_params = {'learning_rate': 0.1, 'n_estimators': 65, 'max_depth': 4, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.85, 'gamma': 0.6, 'reg_alpha': 0, 'reg_lambda': 1}
clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)
optimized_GBM.best_params_

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    2.6s
[Parallel(n_jobs=4)]: Done  80 out of  80 | elapsed:    4.6s finished


[22:47:19] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'reg_alpha': 0.1, 'reg_lambda': 0.05}

In [256]:
###### step 6: learning rate¶

In [428]:
cv_params = {'learning_rate': [0.01, 0.05, 0.07, 0.1, 0.15,0.2]}
other_params = {'learning_rate': 0.1, 'n_estimators': 65, 'max_depth': 4, 'min_child_weight': 1, 'seed': 0,
                    'subsample': 0.8, 'colsample_bytree': 0.85, 'gamma': 0.6, 'reg_alpha': 0.1, 'reg_lambda': 0.05}

In [429]:
clf_xgb = xgb.XGBRegressor(**other_params)
optimized_GBM = GridSearchCV(estimator=clf_xgb, param_grid=cv_params, scoring='r2', cv=5, verbose=1, n_jobs=4) 
optimized_GBM.fit(X_train, y_train)
optimized_GBM.best_params_

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:    1.7s finished


[22:47:39] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


{'learning_rate': 0.1}

##### Build XGBoost Model

In [430]:
clf_xgb = xgb.XGBClassifier(n_estimators=65, max_depth=4, 
                            learning_rate=0.1, subsample=0.8, colsample_bytree=0.85,scale_pos_weight=3.0, 
                             silent=True, nthread=-1, seed=0, missing=None,
                             reg_alpha=0.05, reg_lambda=0.1, 
                             gamma=0.6, min_child_weight=1, 
                             max_delta_step=0,base_score=0.5)

clf_xgb.fit(X_train, y_train)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.85, gamma=0.6,
              learning_rate=0.1, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=65, n_jobs=1,
              nthread=-1, objective='multi:softprob', random_state=0,
              reg_alpha=0.05, reg_lambda=0.1, scale_pos_weight=3.0, seed=0,
              silent=True, subsample=0.8, verbosity=1)

In [431]:
# feature importance 
feature_imp_df = pd.DataFrame(list(zip(clf_xgb.feature_importances_, X_train)))
feature_imp_df.columns = ['feature importance', 'feature']
feature_imp_df = feature_imp_df.sort_values(by='feature importance', ascending=False)
feature_imp_df

,feature importance,feature
40,0.121129,cancellation_policy_super_strict_30
35,0.043140,room_type_Private room
27,0.028210,host_since_2018.0
31,0.026581,host_response_rate_low
3,0.025863,cleaning_fee
38,0.025721,cancellation_policy_moderate
4,0.025540,guests_included
0,0.024833,accommodates
16,0.024561,calculated_host_listings_count
9,0.024201,availability_90


In [446]:
X_test_id = X_test['host_since_2009.0']
X_test = X_test.drop('host_since_2009.0',axis=1)
X_test.insert(18,'host_since_2009.0',X_test_id)

In [447]:
X_test.head()

,accommodates,bedrooms,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_90,availability_365,number_of_reviews,review_scores_rating,review_scores_cleanliness,review_scores_location,review_scores_value,calculated_host_listings_count,reviews_per_month,host_since_2009.0,host_since_2010.0,host_since_2011.0,host_since_2012.0,host_since_2013.0,host_since_2014.0,host_since_2015.0,host_since_2016.0,host_since_2017.0,host_since_2018.0,host_response_time_within a day,host_response_time_within a few hours,host_response_time_within an hour,host_response_rate_low,host_neighbourhood_Prenzlauer Berg,host_identity_verified_t,is_location_exact_t,room_type_Private room,room_type_Shared room,instant_bookable_t,cancellation_policy_moderate,cancellation_policy_strict_14_with_grace_period,cancellation_policy_super_strict_30
0,2,0,200,50,1,0,2,7,1,51,51,10,94,9,10,9,1,1.24,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0,0
1,1,1,500,15,1,18,2,4,150,87,87,2,100,10,10,10,2,0.22,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,1,1,0,0,0,1,0
2,2,0,0,20,1,0,1,1125,7,9,9,3,100,10,10,10,1,0.50,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0
3,4,2,0,40,1,0,3,13,180,75,165,3,100,10,9,9,1,0.21,0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,1,0,0,0,1,0
4,2,1,0,0,1,0,2,15,330,0,0,1,100,8,10,8,2,0.10,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0


In [448]:
# prediction
predictions = pd.DataFrame(clf_xgb.predict(X_test))


In [449]:
predictions.to_csv('predictions.csv')

In [274]:
predictions

,0
0,4.077537
1,3.218876
2,3.912023
3,4.553877
4,3.555348
5,4.174387
6,4.382027
7,3.401197
8,4.382027
9,4.595120
